In [13]:
import json
import asyncio
import nest_asyncio
from typing import List, Dict, Union
from httpx import AsyncClient, Response
from parsel import Selector
from loguru import logger as log
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm.asyncio import tqdm_asyncio
import traceback
import os

# initialize an async httpx client
client = AsyncClient(
    timeout=30.0,
    http2=True,
    headers={
        "Accept-Language": "en-US,en;q=0.9",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Cookie": "intl_splash=false"
    },
    follow_redirects=True
)


def parse_post_comments(response: Response) -> List[Dict]:
    """parse post comments"""

    def parse_comment(parent_selector) -> Dict:
        """parse a comment object"""
        link = parent_selector.xpath("./@data-permalink").get()
        upvotes = parent_selector.xpath(".//span[contains(@class, 'likes')]/@title").get()      
        return {
            "link": "https://www.reddit.com" + link if link else None,
            "commentBody": parent_selector.xpath(".//div[@class='md']/p/text()").get(),
            "upvotes": int(upvotes) if upvotes else None,          
        }

    def parse_replies(what) -> List[Dict]:
        """recursively parse replies"""
        replies = []
        for reply_box in what.xpath(".//div[@data-type='comment']"):
            reply_comment = parse_comment(reply_box)
            child_replies = parse_replies(reply_box)
            if child_replies:
                reply_comment["replies"] = child_replies
            replies.append(reply_comment)
        return replies

    selector = Selector(response.text)
    data = []
    for item in selector.xpath("//div[@class='sitetable nestedlisting']/div[@data-type='comment']"):
        comment_data = parse_comment(item)
        replies = parse_replies(item)
        if replies:
            comment_data["replies"] = replies
        data.append(comment_data)            
    return data


async def scrape_post(url: str, sort: Union["old", "new", "top"]) -> Dict:
    """scrape subreddit post and comment data"""
    response = await client.get(url)
    post_data = {}
    post_link = url if "old.reddit.com" in url else url.replace("www.reddit.com", "old.reddit.com")
    # scrape the comments from the old.reddit version, with the same post URL 
    bulk_comments_page_url = post_link.replace("www", "old") + f"?sort={sort}&limit=5000"
    response = await client.get(bulk_comments_page_url)
    post_data["comments"] = parse_post_comments(response) 
    log.success(f"scraped {len(post_data['comments'])} comments from the post {url}")
    return post_data

In [14]:
###Define the function to figure out the ratio of postive to negative comments in each post
nest_asyncio.apply()

analyzer = SentimentIntensityAnalyzer()
semaphore = asyncio.Semaphore(3)  

def flatten_comments(comments):
    flat = []
    def _recurse(comment):
        flat.append(comment)
        for reply in comment.get("replies", []):
            _recurse(reply)
    for comment in comments:
        _recurse(comment)
    return flat


def load_existing_results(path="team_week_sentiment_long.csv"):
    if os.path.exists(path):
        return pd.read_csv(path)
    else:
        return pd.DataFrame(columns=["Team", "Week", "PositiveRatio", "NegativeRatio", "TotalComments"])


def calculate_sentiment_ratios(comments):
    poscount = 0
    negcount = 0
    for comment in comments:
        body = comment.get("commentBody") or ""
        sentiment = analyzer.polarity_scores(body)
        compound = sentiment["compound"]
        if compound < -0.05:
            negcount += 1
        elif compound > 0.05:
            poscount += 1
    total = poscount + negcount
    if total == 0:
        return 0.0, 0.0, 0
    return poscount / total, negcount / total, total

async def analyze_from_csv(csv_path: str):
    df = pd.read_csv(csv_path)
    df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="URL")
    df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
    df_melted = df_melted.dropna(subset=["URL"])

    existing_df = load_existing_results()

    successful_df = existing_df[
    (existing_df["PositiveRatio"] > 0.0) | 
    (existing_df["NegativeRatio"] > 0.0) | 
    (existing_df["TotalComments"] > 0)
]

    existing_keys = set(zip(successful_df["Team"], successful_df["Week"]))

# Filter the melted dataframe to only those not already scraped
    df_to_scrape = df_melted[~df_melted.apply(lambda row: (row["Team"], row["Week"]) in existing_keys, axis=1)]
    print(df_to_scrape.head())


    results = []

    async def process_row(row):
        async with semaphore:
            try:
                team = row["Team"]
                week = row["Week"]
                url = row["URL"]
                print(f"Processing: Team={team}, Week={week}, URL={url}")
                post_data = await scrape_post(url, sort="top")
                comments = flatten_comments(post_data["comments"])
                pos, neg, total = calculate_sentiment_ratios(comments)
                return {
                    "Team": team,
                    "Week": week,
                    "PositiveRatio": pos,
                    "NegativeRatio": neg,
                    "TotalComments": total
                }
            except Exception as e:
                print(f"Failed: {team} {week} -> {e}")
                traceback.print_exc()
                return {
                    "Team": team,
                    "Week": week,
                    "PositiveRatio": 0.0,
                    "NegativeRatio": 0.0,
                    "TotalComments": 0
                }


    tasks = [process_row(row) for _, row in df_to_scrape.iterrows()]
    results = await tqdm_asyncio.gather(*tasks)

    # After getting `results` from scraping:
    new_results_df = pd.DataFrame(results)

    # Combine with old data, drop duplicates
    combined_df = pd.concat([existing_df, new_results_df], ignore_index=True)
    combined_df.drop_duplicates(subset=["Team", "Week"], keep="last", inplace=True)

    # Save updated file
    combined_df.to_csv("team_week_sentiment_long.csv", index=False)
    return combined_df


In [16]:
### Get the dataset of sentiment
result_df = asyncio.run(analyze_from_csv("NFL 2024-25 Subreddit - Sheet1.csv"))
print(result_df.head())

Empty DataFrame
Columns: [Team, Week, URL]
Index: []


0it [00:00, ?it/s]

                 Team    Week  PositiveRatio  NegativeRatio  TotalComments
0   Arizona Cardinals  Week 1       0.560079       0.439921           1007
1     Atlanta Falcons  Week 1       0.492991       0.507009            428
2   Carolina Panthers  Week 1       0.565161       0.434839            775
3  Cincinnati Bengals  Week 1       0.529837       0.470163            553
4    Cleveland Browns  Week 1       0.585324       0.414676            586


In [ ]:
### Merge to get the full dataset
result_df = pd.read_csv('team_week_sentiment_long.csv')


df = pd.read_csv('NFL 2024-25 Div Games - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Div")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Div"])
merged_df = pd.merge(result_df, df_melted, on=["Team", "Week"], how="left")

df = pd.read_csv('NFL 2024-25 Home - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Home")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Home"])
merged_df = pd.merge(merged_df, df_melted, on=["Team", "Week"], how="left")



df = pd.read_csv('NFL 2024-25 Margins - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Margin of Victory")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Margin of Victory"])
merged_df = pd.merge(merged_df, df_melted, on=["Team", "Week"], how="left")




df = pd.read_csv('NFL 2024-25 Opps - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Opponent")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Opponent"])
merged_df = pd.merge(merged_df, df_melted, on=["Team", "Week"], how="left")



df = pd.read_csv('NFL 2024-25 Playoffs - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Playoff")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Playoff"])
merged_df = pd.merge(merged_df, df_melted, on=["Team", "Week"], how="left")



df = pd.read_csv('NFL 2024-25 Primetime - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Prime Time")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Prime Time"])
merged_df = pd.merge(merged_df, df_melted, on=["Team", "Week"], how="left")


df = pd.read_csv('NFL 2024-25 W-L - Sheet1.csv')
df_melted = df.melt(id_vars=["Unnamed: 0"], var_name="Week", value_name="Win-Lose")
df_melted = df_melted.rename(columns={"Unnamed: 0": "Team"})
df_melted = df_melted.dropna(subset=["Win-Lose"])
merged_df = pd.merge(merged_df, df_melted, on=["Team", "Week"], how="left")

merged_df['Week'] = merged_df['Week'].replace('WIld Card', 'Wild Card')
print(merged_df.head())



merged_df.to_csv("NFL 2024-25 Full.csv", index=False)


                 Team    Week  PositiveRatio  NegativeRatio  TotalComments  \
0   Arizona Cardinals  Week 1       0.560079       0.439921           1007   
1     Atlanta Falcons  Week 1       0.492991       0.507009            428   
2   Carolina Panthers  Week 1       0.565161       0.434839            775   
3  Cincinnati Bengals  Week 1       0.529837       0.470163            553   
4    Cleveland Browns  Week 1       0.585324       0.414676            586   

   Div Home  Margin of Victory              Opponent Playoff Prime Time  \
0   No   No               -6.0         Buffalo Bills      No         No   
1   No  Yes               -8.0   Pittsburgh Steelers      No         No   
2  Yes   No              -37.0    New Orleans Saints      No         No   
3   No  Yes               -6.0  New England Patriots      No         No   
4   No  Yes              -16.0        Dallas Cowboys      No         No   

  Win-Lose  
0        L  
1        L  
2        L  
3        L  
4        L  


In [ ]:
####Code to see individual comments

nest_asyncio.apply()

async def run():
    post_data = await scrape_post(
        url = "https://www.reddit.com/r/eagles/comments/1ilwnz0/post_game_thread_super_bowl_champions_the_eagles/",
        sort="top"
    )
    with open("post.json", "w", encoding="utf-8") as file:
        json.dump(post_data, file, indent=2, ensure_ascii=False)


if __name__ == "__main__":
    asyncio.run(run())


import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the data
with open("post.json", "r", encoding="utf-8") as f:
    data = json.load(f)

comments = data["comments"]


# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Flatten all comments including replies
def flatten_comments(comments):
    flat = []

    def _recurse(comment):
        flat.append(comment)
        if "replies" in comment:
            for reply in comment["replies"]:
                _recurse(reply)

    for comment in comments:
        _recurse(comment)
    
    return flat

flat_comments = flatten_comments(comments)


# Run sentiment analysis
for comment in flat_comments:
    body = comment.get("commentBody") or ""
    sentiment = analyzer.polarity_scores(body)
    comment["sentiment"] = sentiment 


# Example: print the results
for comment in flat_comments:
    print(f"Comment: {comment['commentBody']}")
    print(f"Sentiment: {comment['sentiment']}")
    print(f"Upvote: {comment['upvotes']}")
    print('-' * 60)



poscount = 0
negcount = 0
for comment in flat_comments:
    compound = comment['sentiment']['compound']
    if compound < 0:
        negcount += 1
    if compound >= 0.05:
        poscount += 1

total = poscount +negcount
pos = poscount/total
neg = negcount/total
print(pos)
print(neg)
print(total)


2025-05-09 17:10:22.602 | SUCCESS  | __main__:scrape_post:73 - scraped 434 comments from the post https://www.reddit.com/r/eagles/comments/1ilwnz0/post_game_thread_super_bowl_champions_the_eagles/


Comment: We are World Champs! Time to celebrate our second Lombardi! 
Sentiment: {'neg': 0.0, 'neu': 0.53, 'pos': 0.47, 'compound': 0.7955}
Upvote: None
------------------------------------------------------------
Comment: Jake Elliott made 4 FGs and then re-made 3 of them from 5 yards further away as a big "fuck you" to all of us and I love that for him.
Sentiment: {'neg': 0.095, 'neu': 0.79, 'pos': 0.114, 'compound': 0.1779}
Upvote: 751
------------------------------------------------------------
Comment: Good. He showed up when it mattered most and hit one from 50 too
Sentiment: {'neg': 0.0, 'neu': 0.818, 'pos': 0.182, 'compound': 0.4404}
Upvote: 147
------------------------------------------------------------
Comment: Whatever the fuck ritual he was doing 5 hours before the game absolutely paid off. Good to have him back to his usual self for the biggest game
Sentiment: {'neg': 0.107, 'neu': 0.796, 'pos': 0.097, 'compound': -0.0865}
Upvote: 81
--------------------------------------